### Instalações

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!pip install pandas
import pandas as pd
import scipy.stats
import sklearn
import numpy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")

[nltk_data] Downloading package punkt to /Users/lucelia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#!pip3 install numpy --upgrade
#!pip install numpy-1.22.4

In [4]:

#!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Embedding, TimeDistributed, Bidirectional,GlobalMaxPooling1D

#!pip install livelossplot
from livelossplot.tf_keras import PlotLossesCallback

#!pip install seqeval
from seqeval.metrics import f1_score, classification_report, precision_score, recall_score, accuracy_score
from seqeval.scheme import IOB2

#! pip install plot_keras_history
from plot_keras_history import plot_history

#!pip install tensorflow_addons
from tensorflow_addons.text.crf_wrapper import CRFModelWrapper
from tensorflow_addons.layers.crf import CRF
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#!pip install sklearn_crfsuite
from sklearn_crfsuite import CRF
from seqeval.scheme import IOB2

2023-07-21 13:26:34.581983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Dados

In [5]:
import pandas as pd

#df = pd.read_csv('/content/drive/MyDrive/Experimentos/Alice/DODFCorpus_contratos_licitacoes_v2.csv')
df = pd.read_csv('./CSVs/V2/DODFCorpus_contratos_licitacoes_v2.csv', dtype=str)

df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)

##Insere um espaço entre as entidade e insere um espaco apos :  : -> Processo:0...
#df['texto'] = df['texto'].str.replace(r'([A-Za-z]:)[0-9]', r'\1 ', regex=True)

In [6]:
##Insere um espaço entre as entidade e insere um espaco apos :  : -> Processo:0...
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [7]:
df['texto']= df['texto'].map(correct_space_before_numeric_entities)

In [8]:
#df

In [9]:
df.tipo_rel.unique()

array(['REL_AVISO_LICITACAO', 'REL_SUSPENSAO_LICITACAO',
       'REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_ANUL_REVOG_LICITACAO', 'REL_EXTRATO_CONVENIO'], dtype=object)

In [10]:
#Concatena o id_ato com o id_dodf
df['id_ato'] = df['id_dodf'] + '-' + df['id_rel']
#data

In [11]:
#Lista o tipo de relatório
result =df.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))

tipo_rel
REL_EXTRATO_CONTRATO        1734
REL_ADITAMENTO_CONTRATO     1551
REL_AVISO_LICITACAO          639
REL_SUSPENSAO_LICITACAO       82
REL_ANUL_REVOG_LICITACAO      52
REL_EXTRATO_CONVENIO          32
Name: id_ato, dtype: int64


In [12]:
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_ADITAMENTO_CONTRATO'].reset_index(drop=True)
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_SUSPENSAO_LICITACAO'].reset_index(drop=True)
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_EXTRATO_CONVENIO'].reset_index(drop=True)
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_ANUL_REVOG_LICITACAO'].reset_index(drop=True)
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_AVISO_LICITACAO'].reset_index(drop=True)
#tipo_ato = df.loc[df['tipo_rel'] == 'REL_EXTRATO_CONTRATO'].reset_index(drop=True)

#tipo_ato = df.query("tipo_rel == 'REL_ANUL_REVOG_LICITACAO'").reset_index(drop=True)
##tipo_ato = df.query("tipo_rel == 'REL_ADITAMENTO_CONTRATO'").reset_index(drop=True)
tipo_ato = df.query("tipo_rel == 'REL_SUSPENSAO_LICITACAO'").reset_index(drop=True)
#tipo_ato = df.query("tipo_rel == 'REL_AVISO_LICITACAO'").reset_index(drop=True)
#tipo_ato = df.query("tipo_rel == 'REL_EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato = df.query("tipo_rel == 'REL_EXTRATO_CONVENIO'").reset_index(drop=True)

#tipo_ato.to_csv("./CSVs/V1/REL_AVISO_LICITACAO.csv")
#tipo_ato

In [13]:
#Lista as entidades do tipo de ato
result = tipo_ato.groupby('tipo_ent')['id_ato'].nunique()
print(result)
#print(result.sort_values(ascending=False))

tipo_ent
AVISO_SUSPENSAO_LICITACAO    82
decisao_tcdf                 17
modalidade_licitacao         75
nome_responsavel             81
numero_licitacao             79
objeto_licitacao             71
orgao_licitante              70
processo_gdf                 63
Name: id_ato, dtype: int64


In [14]:
#cria um novo dataset apenas com o tipo de entidade EXTATO_CONTRATO (TEXTO)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'AVISO_ANUL_REV_LICITACAO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_ADITAMENTO_CONTRATUAL'").reset_index(drop=True)
tipo_ato1 = tipo_ato.query("tipo_ent == 'AVISO_SUSPENSAO_LICITACAO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONVENIO'").reset_index(drop=True)
#tipo_ato1

In [15]:
#criando novo dataframe somente com os textos
data = tipo_ato1['texto'].reset_index(drop=True)
#data.to_csv("./CSVs/V2/REL_ANUL_REVOG_LICITACAO.csv")
data

0     AVISO DE SUSPENSAO PREGAO ELETRONICO POR SRP N...
1     AVISO DE SUSPENSAO CONCORRENCIA No 02/2018 O M...
2     AVISO DE SUSPENSAO DE LICITACAO PREGAO ELETRON...
3     AVISO DE SUSPENSAO PREGAO ELETRONICO No 07/201...
4     AVISO DE SUSPENSAO DE LICITACAO LICITACAO CEB-...
                            ...                        
77    AVISO DE SUSPENSÃO E NOVA DATA DO PREGÃO ELETR...
78    AVISO DE SUSPENSÃO E NOVA DATA DO PREGÃO ELETR...
79    AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO POR SRP N...
80    AVISO DE SUSPENSÃO CONCORRÊNCIA PARA CONCESSÃO...
81    AVISO DE REALIZAÇÃO DE EVENTO DE SUSPENSÃO PRE...
Name: texto, Length: 82, dtype: object

In [16]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df = []

for i in data:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    aux = ' '.join(i.split())
    df.append(aux)
    

In [17]:
df

['AVISO DE SUSPENSAO PREGAO ELETRONICO POR SRP No 459/2018 - UASG 926119 A Pregoeira da Central de Compras/SUAG, da Secretaria de Estado de Saude do Distrito Federal, comunica a suspensao da licitacao supracitada, por acatar a impugnacao apresentada. LARISSA ALVIM DE OLIVEIRA',
 'AVISO DE SUSPENSAO CONCORRENCIA No 02/2018 O METRO-DF, por meio do Presidente da Comissao Especial de Licitacao, comunica a suspensao sine die da Concorrencia no 02/2018, cujo objeto e a contratacao de empresa para execucao de Obras Civis para Expansao da Linha 1 - Trecho Samambaia do METRO-DF, conforme processo n.o 00097-00004868/2018- 01, cuja data da sessao estava prevista para o dia 21/11/2018. A suspensao se da por forca da Decisao no 5531/2018 - TCDF. ANDERSON LUIZ SENNA COSTA Presidente da Comissao',
 'AVISO DE SUSPENSAO DE LICITACAO PREGAO ELETRONICO No 001-S001061/2018 A CEB DISTRIBUICAO S.A., por sua Comissao Permanente de Licitacao - CPL, situada no SIA Setor de Areas Publicas, Lote "C", bloco "H", 

## Aplicando a Supervisão Fraca

In [18]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA

#Para rodar aditamento de contrato 
#import dodfSkweak_Aditamento_2
#skw = dodfSkweak_Aditamento_2.SkweakAditamento(df)

#Para rodar Aviso Licitação
#import dodfSkweak_Aviso_Licitacao
#skw = dodfSkweak_Aviso_Licitacao.SkweakAvisoLicitacao(df)

#Para rodar Suspensao 
import dodfSkweak_Suspensao_1
skw = dodfSkweak_Suspensao_1.SkweakSuspensao(df)

#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog_1
#skw = dodfSkweak_Anul_Revog_1.SkweakAnulRevog(df)

#Para rodar extrato de contrato
#import dodfSkweak_Contrato
#skw = dodfSkweak_Contrato.SkweakContratos(df)

#Para rodar Convenio
#import dodfSkweak_Convenio
#skw = dodfSkweak_Convenio.SkweakConvenio(df)

skw.apply_label_functions()
skw.train_HMM_Dodf()


/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Finished E-step with 75 documents
Starting iteration 2


         1       -1616.6532             +nan


Finished E-step with 75 documents
Starting iteration 3


         2       -1558.3614         +58.2918


Finished E-step with 75 documents
Starting iteration 4
Finished E-step with 75 documents


         3       -1557.3929          +0.9685
         4       -1557.2154          +0.1776


Write to ./data/suspensao.spacy...done


In [19]:
#List os tokens
#skw.list_spans_all()
skw.list_spans_specific(1)

[CONCORRENCIA, 02/2018, Concorrencia, 02/2018, 00097-00004868/2018- 01]


In [20]:
#Imprimi o Dataset gerado pela supervisão 
data_label=skw.get_hmm_dataframe()
data_label
# converting to CSV file
#data_label.to_csv("./CSVs/V2/supervisao_fraca/CONVENIO.csv")

,processo_gdf,modalidade_licitacao,numero_licitacao,text,labels
0,,PREGAO ELETRONICO,459/2018,AVISO DE SUSPENSAO PREGAO ELETRONICO POR SRP N...,O O O B-modalidade_licitacao I-modalidade_lici...
1,00097-00004868/2018- 01,Concorrencia,02/2018,AVISO DE SUSPENSAO CONCORRENCIA No 02/2018 O M...,O O O B-modalidade_licitacao O B-numero_licita...
2,,PREGAO ELETRONICO,001-S001061/2018,AVISO DE SUSPENSAO DE LICITACAO PREGAO ELETRON...,O O O O O B-modalidade_licitacao I-modalidade_...
3,00055-00034558/2019-90,PREGAO ELETRONICO,07/2019,AVISO DE SUSPENSAO PREGAO ELETRONICO No 07/201...,O O O B-modalidade_licitacao I-modalidade_lici...
4,,,,AVISO DE SUSPENSAO DE LICITACAO LICITACAO CEB-...,O O O O O O O O O O O O O O O O O O O O O O O ...
...,...,...,...,...,...
77,1.744/2021,PREGÃO ELETRÔNICO,12/2022,AVISO DE SUSPENSÃO E NOVA DATA DO PREGÃO ELETR...,O O O O O O O B-modalidade_licitacao I-modalid...
78,130/2022,PREGÃO ELETRÔNICO,16/2022,AVISO DE SUSPENSÃO E NOVA DATA DO PREGÃO ELETR...,O O O O O O O B-modalidade_licitacao I-modalid...
79,0060-00353385/2020-43),PREGÃO ELETRÔNICO,241/2021,AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO POR SRP N...,O O O B-modalidade_licitacao I-modalidade_lici...
80,04003-00000200/2020-18 (,CONCORRÊNCIA,,AVISO DE SUSPENSÃO CONCORRÊNCIA PARA CONCESSÃO...,O O O B-modalidade_licitacao O O O O O O O O O...


In [21]:
#Aplica o IOB com spacy tokenizer
corpus = skw.get_IOB()
#corpus

In [22]:
#Separa Tokens e os labels
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        classes=debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y) 

In [23]:
#termos_skweak

In [24]:
debug

{'B-modalidade_licitacao',
 'B-numero_licitacao',
 'B-processo_gdf',
 'I-modalidade_licitacao',
 'I-processo_gdf',
 'O'}

In [25]:
act_name = 'SUSPENSAO_LICITACAO'
act_name

'SUSPENSAO_LICITACAO'

# Processando os splits

In [26]:
#!unzip train.zip

In [27]:
##Carrega os split de teste
import os
import pandas as pd
import numpy as np
import pickle

test_splits = {}
for file_name in os.listdir('./CSVs/V2/split/CISTI/WS/teste_ws/'):
  #print("nome do arquivo", file_name)
  
  # Ignora os arquivos .csv
  if file_name[-3:] == 'csv':
    continue

  #print("file name",file_name)
  # Descarta a terminação "_n_x_test.txt" do final do nome do arquivo
  act_name = "_".join(file_name.split("_")[0:-3])
  #print(file_name)
  
  split_number = int(file_name.split("_")[-3])

  x_or_y = file_name.split("_")[-2]
  
  if act_name not in test_splits.keys():
    test_splits[act_name] = {}

  if split_number not in test_splits[act_name].keys():
    test_splits[act_name][split_number] = {}

  # Se for x
  if x_or_y == 'x':
    with open(os.path.join('./CSVs/V2/split/CISTI/WS/teste_ws/', file_name), 'rb') as f:
      test_splits[act_name][split_number]['x'] = pickle.load(f)
    continue

  # Se for y
  with open(os.path.join('./CSVs/V2/split/CISTI/WS/teste_ws/', file_name), 'rb') as f:
      test_splits[act_name][split_number]['y'] = pickle.load(f)
  

In [28]:
test_splits.keys()

dict_keys(['AVISO_LICITACAO', 'ADITAMENTO_CONTRATO', 'SUSPENSAO_LICITACAO', 'ANUL_REV_LICITACAO', 'EXTRATO_CONTRATO', 'EXTRATO_CONVENIO'])

In [29]:
max_length = 400

### Ajustes das labels e dicionários

In [30]:
def remove_wrong_tags(label_list):
  for label in label_list:
    for idx,w in enumerate(label):
      if w in ['B-11','B-12','B-50', 'B-60', 'I-2', '50']:
        label[idx] = 'O'

In [31]:
map_labels = {'cnpj_entidade_convenente':'cnpj_entidade_contratada', 
'cnpj_orgao_concedente':'cnpj_orgao_contratante', 
'data_assinatura_convenio':'data_assinatura_contrato', 
'entidade_convenente':'entidade_contratada',
'numero_convenio':'numero_contrato',
'objeto_convenio':'objeto_contrato',
'orgao_concedente':'orgao_contratante',
'valor_convenio':'valor_contrato',
'vigencia_convenio':'vigencia_contrato'}

def transform_labels(targets):
  for text in targets:
    for i, word in enumerate(text):
        if word[2:] in map_labels:
            text[i] = text[i].replace(word[2:], map_labels[word[2:]])

In [32]:
#if act_name == 'Extrato_Contrato':
#    transform_labels(labels_skweak)

In [33]:
remove_wrong_tags(labels_skweak)

### Transformação dos dados

In [34]:
words = set()

for act in termos_skweak:
    for word in act:
        words.add(word)
#convertendo o set em uma lista
words = list(words)

words.append("ENDPAD")
words.append("UNK")

words_amt = len(words)

In [35]:
tags = set()

for label in labels_skweak:
    for tag in label:
        tags.add(tag)
tags = list(tags)
tags_amt = len(tags)

In [36]:
def word2features(sent, i):
    word = sent[i]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),        
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [37]:
for i in range(0,5):
  inputs = [sent2features(s) for s in termos_skweak]

In [38]:
#termos_skweak

In [39]:
targets = labels_skweak

### Funções auxiliares

In [40]:
def original_labels(sent):
    for i, word in enumerate(sent):
        if word[2:] in map_labels.values():
                original = [key for key,value in map_labels.items() if value==word[2:]]
                sent[i] = sent[i].replace(word[2:], original[0])

    
    return sent

### Kfold

In [41]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

In [42]:
#targets

In [43]:
act_name 

'ANUL_REV_LICITACAO'

### CRF

In [47]:
acc = []
loss = []
f1 = []
reports = []
fold = 0

#for index in range(0,5):
for train,test in kfold.split(inputs, targets):
  
  x_train = np.array(inputs)[train.astype(int)]
  #x_test = np.array(inputs)[test.astype(int)]

  y_train = np.array(targets)[train.astype(int)]
  #y_test = np.array(targets)[test.astype(int)]
  
  #x_train = act_splits[act_name][index]['x']
  #y_train = act_splits[act_name][index]['y']
  
  x_test = test_splits[act_name][fold]['x']
  y_test = test_splits[act_name][fold]['y']

  
  crf = CRF(
    algorithm='lbfgs', 
    c1=	0.1181, 
    c2= 0.0069,  
    max_iterations=70, 
    all_possible_transitions=True, verbose=1)


  crf.fit(X=x_train, y=y_train)

  y_pred = crf.predict(x_test)


  ''' if act_name == 'Extrato_Contrato':
    for i, sent in enumerate(x_test):
        all_values = [value for elem in sent for value in elem.values()]
        if 'convenio' in sent or 'convênio' in all_values:
          y_pred[i] = original_labels(y_pred[i])
          y_test[i] = original_labels(y_test[i]) '''
  
  
  f1.append(f1_score(y_test, y_pred))

  reports.append(classification_report(y_test,y_pred))

  acc.append(accuracy_score(y_test, y_pred))

  r = classification_report(y_test,y_pred, output_dict=True, mode='strict', scheme=IOB2)
  #name = act_name + "_numero" + '_f' + str(fold) +".npy"
  
  name = act_name + '_f' + str(fold) +".npy"
  np.save('./Results/V2/CISTI/27_06/WS_CRF/'+name, r)
  fold = fold + 1
  reports.append(r)
  

SyntaxError: invalid syntax (2727898683.py, line 43)

In [45]:
y_pred

[['O',
  'O',
  'O',
  'B-modalidade_licitacao',
  'I-modalidade_licitacao',
  'O',
  'B-processo_gdf',
  'I-processo_gdf',
  'O',
  'O',
  'B-processo_gdf',
  'I-processo_gdf',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-processo_gdf',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [ ]:
  r = classification_report(y_test,y_pred, output_dict=False, mode='strict', scheme=IOB2)

  name = 'crf_' + act_name + '_f' + str(index) +".npy"
  np.save('./Results/V2/CISTI/'+name, r)


NameError: name 'index' is not defined

In [ ]:
f1

[0.20689655172413793,
 0.18823529411764706,
 0.20224719101123595,
 0.14634146341463414,
 0.16]

In [ ]:
print(reports[0])

                                  precision    recall  f1-score   support

codigo_licitacao_sistema_compras       0.00      0.00      0.00        89
         data_abertura_licitacao       0.00      0.00      0.00       131
            modalidade_licitacao       0.89      0.84      0.86       142
                nome_responsavel       0.00      0.00      0.00       125
                numero_licitacao       0.98      0.79      0.88       144
                objeto_licitacao       0.00      0.00      0.00       144
                 orgao_licitante       0.00      0.00      0.00        83
                    processo_gdf       0.59      0.74      0.66       117
                 sistema_compras       0.00      0.00      0.00       147
                     tipo_objeto       0.00      0.00      0.00        35
      valor_estimado_contratacao       0.00      0.00      0.00       107

                       micro avg       0.81      0.25      0.38      1264
                       macro avg    

In [ ]:
np.mean(f1)

0.9070432618274509

In [ ]:
act_name

'EXTRATO_CONVENIO'

In [ ]:
np.std(f1)

0.044205670842667445